# 폴리움 군집 시각화 

## 폴리움 시각화 데이터 준비

In [13]:
from data_preparation import get_cluster_df, get_data 
clustering_df = get_cluster_df() 
previous_df_for_join = get_data() 

In [14]:
def filter_by_service(df, column_name, service_type): 
    return df[df[column_name] == service_type]

def drop_columns(df, columns_to_drop): 
    return df.drop(columns_to_drop, axis=1) 

In [15]:
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

def perform_clustering(dataframe, n_components=4, n_clusters=6):
    minmax_scaled = MinMaxScaler().fit_transform(dataframe)
    
    dim = TruncatedSVD(n_components=n_components).fit_transform(minmax_scaled)
    
    model = KMeans(n_clusters=n_clusters, init='k-means++', algorithm='auto', max_iter=300, random_state=0)
    model.fit(dim)
    
    dataframe['cluster'] = model.labels_
    
    return dataframe

In [16]:
villa_clustering_df = filter_by_service(clustering_df, 'service_type_빌라', 1)
oneroom_clustering_df = filter_by_service(clustering_df, 'service_type_원룸', 1)
officetel_clustering_df = filter_by_service(clustering_df, 'service_type_오피스텔', 1)

villa_clustering_df = drop_columns(villa_clustering_df, ['service_type_빌라','service_type_오피스텔','service_type_원룸'])
oneroom_clustering_df = drop_columns(oneroom_clustering_df, ['service_type_빌라','service_type_오피스텔','service_type_원룸'])
officetel_clustering_df = drop_columns(officetel_clustering_df, ['service_type_빌라','service_type_오피스텔','service_type_원룸'])

In [17]:
villa_cluster = perform_clustering(villa_clustering_df)
oneroom_cluster = perform_clustering(oneroom_clustering_df)
officetel_cluster = perform_clustering(officetel_clustering_df)

In [20]:
villa_cluster = villa_cluster.combine_first(filter_by_service(previous_df_for_join, 'service_type', '빌라'))
oneroom_cluster = oneroom_cluster.combine_first(filter_by_service(previous_df_for_join, 'service_type', '원룸'))
officetel_cluster = officetel_cluster.combine_first(filter_by_service(previous_df_for_join, 'service_type', '오피스텔'))

## 폴리움 시각화 

In [26]:
import folium
from folium.plugins import MarkerCluster
from folium import Marker, Popup, IFrame, Circle

In [39]:
def adjust_cluster(row):
    if row['service_type'] == '원룸':
        return row['cluster'] + 6
    elif row['service_type'] == '오피스텔':
        return row['cluster'] + 12
    else:
        return row['cluster']

In [40]:
oneroom_cluster['cluster'] = oneroom_cluster.apply(adjust_cluster, axis=1)
officetel_cluster['cluster'] = officetel_cluster.apply(adjust_cluster, axis=1)

In [42]:
def color_select(row, num_clusters):
    cluster = row['cluster']
    
    colors = ['red', 'orange', 'yellow', 'green', 'blue', 'violet', 'indigo', 'black',
              'pink', 'brown', 'purple', 'gray', 'olive', 'cyan', 'magenta', 'lime',
              'teal', 'navy']
    
    return colors[cluster]

In [43]:
def popup_text(row):
    pop_up_text = f"""
    <div style="
    font-size:14px;
    line-height: 1.7;
    color: #333;
    background-color: #f9f9f9;
    padding: 10px;
    border-radius: 4px;
    font-family: Arial, sans-serif;
    background-size: cover;
    ">
    <b style="color: #ff4500;">매물id:</b> {row['id']} <br>
    <b style="color: #ff4500;">주소:</b> {row['address1']} {row['address2']} <br> 
    <b style="color: #ff4500;">월세(조정된):</b> {row['rent_adjusted']} <br> 
    <b style="color: #ff4500;">크기:</b> {row['size_m2']}m2 <br> 
    <b style="color: #ff4500;">엘레베이터:</b> {row['elevator']} <br> 
    <b style="color: #ff4500;">주차:</b> {row['parking']} <br> 
    <b style="color: #ff4500;">군집:</b> {row['cluster']+1} <br>
    
    {'<b style="color: #ff4500;">외국인 비율 자취 비율이 높고 안전지수가 높은 군집 </b>' if row['cluster'] == 0 else ''} 
    {'<b style="color: #ff4500;">건물 설비가 좋지 않고 낮은 층수지만 임대료가 저렴한 군집 </b>' if row['cluster'] == 1 else ''} 
    {'<b style="color: #ff4500;">건물 설비가 좋지만 교통이 좋지 않은 군집</b>' if row['cluster'] == 2 else ''} 
    {'<b style="color: #ff4500;">노령화지수가 높고 건물 설비가 좋지 않은 임대료가 낮은 군집</b>' if row['cluster'] == 3 else ''} 
    {'<b style="color: #ff4500;">건물 설비가 좋고 주거 환경 좋지만 평수가 좁은데 비교적 집값이 높은 군집</b>' if row['cluster'] == 4 else ''} 
    {'<b style="color: #ff4500;">남녀성비가 높고 안전지수가 낮고 교통이 좋지 않지만 임대료가 낮은 군집</b>' if row['cluster'] == 5 else ''} 
    
    {'<b style="color: #ff4500;">건물 설비가 좋지만 주거 환경은 안좋고 교통이 좋지 않은 군집</b>' if row['cluster'] == 6 else ''} 
    {'<b style="color: #ff4500;">자취비율이 높고 안전지수가 높지만 작은 평수의 매물인 군집 </b>' if row['cluster'] == 7 else ''} 
    {'<b style="color: #ff4500;">교통이 좋지 않고 안전하지 않지만 좋은 건물 설비에 비해 집값이 저렴한 군집</b>' if row['cluster'] == 8 else ''} 
    {'<b style="color: #ff4500;">저층의 작은 평수의 매물이 많지만 교통이 편리하고 집값이 저렴한 군집</b>' if row['cluster'] == 9 else ''} 
    {'<b style="color: #ff4500;">인구 밀도가 높고 작은 평수의 매물이지만 편의시설이랑 가까운 군집</b>' if row['cluster'] == 10 else ''} 
    {'<b style="color: #ff4500;">안전지수가 높고 교통이 좋지만 임대료가 비싼 군집</b>' if row['cluster'] == 11 else ''} 
    
    {'<b style="color: #ff4500;">집값이 비교적 저렴하지만 교통이 좋지 않은 군집</b>' if row['cluster'] == 12 else ''} 
    {'<b style="color: #ff4500;">모든 특징이 오피스텔 군집과 비슷해서 되게 무난한 군집</b>' if row['cluster'] == 13 else ''} 
    {'<b style="color: #ff4500;">노령화지수와 안전지수가 높고 교통이 좋지만 임대료가 비싼 군집 </b>' if row['cluster'] == 14 else ''} 
    {'<b style="color: #ff4500;">다른 특징들은 평균적으로 무난하지만 안전지수가 낮은 군집 </b>' if row['cluster'] == 15 else ''} 
    {'<b style="color: #ff4500;">큰 평수의 매물이 많지만 임대료가 비싼 군집 </b>' if row['cluster'] == 16 else ''} 
    {'<b style="color: #ff4500;">자취비율이 높고 인구가 밀집되어 교통이 좋지만 임대료가 비싼 군집</b>' if row['cluster'] == 17 else ''} 
    </div>
    """
    return pop_up_text

In [38]:
def adjust_cluster(row):
    if row['service_type'] == '원룸':
        return row['cluster'] + 6
    elif row['service_type'] == '오피스텔':
        return row['cluster'] + 12
    else:
        return row['cluster']

In [33]:
def create_map(dataframe, num_clusters, location=[37.55, 127.08], zoom_start=15):
    m = folium.Map(location=location, zoom_start=zoom_start)

    for i, row in dataframe.iterrows():
        address = dataframe['address1'][i] + dataframe['address2'][i]
        iframe = IFrame(popup_text(row))
        popup = Popup(iframe, min_width=500, max_width=500)
        Circle(location = [row['y_w84'], row['x_w84']],
               popup=popup, color=color_select(row, num_clusters), radius=50, tooltip=address
              ).add_to(m)
    
    return m

In [ ]:
import pandas as pd 
villa_oneroom_officetel_combined = pd.concat([villa_cluster, oneroom_cluster, officetel_cluster])

combined_map = create_map(villa_oneroom_officetel_combined, 18)
combined_map

In [ ]:
villa_map = create_map(villa_cluster, 6) 
villa_map 

In [ ]:
oneroom_map = create_map(oneroom_cluster, 6)
oneroom_map

In [ ]:
officetel_map = create_map(officetel_cluster, 6) 
officetel_map 

In [55]:
combined_map.save("combined_folium.html")
villa_map.save("villa_folium.html") 
oneroom_map.save("oneroom_folium.html") 
officetel_map.save("officetel_folium.html") 